In [102]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder

In [103]:
df = pd.read_csv("dataset_mood_smartphone.csv")
df['time'] = pd.to_datetime(df['time'])
df['date'] = df['time'].dt.date
df.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

df

,index,id,time,variable,value,date
0,1,AS14.01,2014-02-26 13:00:00.000,mood,6.000,2014-02-26
1,2,AS14.01,2014-02-26 15:00:00.000,mood,6.000,2014-02-26
2,3,AS14.01,2014-02-26 18:00:00.000,mood,6.000,2014-02-26
3,4,AS14.01,2014-02-26 21:00:00.000,mood,7.000,2014-02-26
4,5,AS14.01,2014-02-27 09:00:00.000,mood,6.000,2014-02-27
...,...,...,...,...,...,...
376907,2770399,AS14.30,2014-04-11 07:51:16.948,appCat.weather,8.032,2014-04-11
376908,2772465,AS14.30,2014-04-19 11:00:32.747,appCat.weather,3.008,2014-04-19
376909,2774026,AS14.30,2014-04-26 10:19:07.434,appCat.weather,7.026,2014-04-26
376910,2774133,AS14.30,2014-04-27 00:44:48.450,appCat.weather,23.033,2014-04-27


In [104]:
df_filtered = df[df['variable'].isin(['sms', 'call'])]

# Aggregate counts by individual and event type
counts = df_filtered.groupby(['id', 'variable'])['value'].size().reset_index(name='count')

# Determine the IQR for outlier detection
Q1 = counts['count'].quantile(0.25)
Q3 = counts['count'].quantile(0.75)
IQR = Q3 - Q1
outliers_threshold_low = Q1 - 1.5 * IQR
outliers_threshold_high = Q3 + 1.5 * IQR

# Identify outliers
outliers = counts[(counts['count'] < outliers_threshold_low) | (counts['count'] > outliers_threshold_high)]

In [105]:
# Filter for 'sms' and 'call' events
df_sms_call = df[df['variable'].isin(['sms', 'call'])]

# Group by individual ('id'), variable, and date to count daily events
daily_counts = df_sms_call.groupby(['id', 'variable', df_sms_call['time'].dt.date]).size().reset_index(name='daily_count')


In [106]:
# Calculate IQR for daily counts
Q1 = daily_counts['daily_count'].quantile(0.25)
Q3 = daily_counts['daily_count'].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as those beyond 1.5 times the IQR from the quartiles
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outlier rows based on daily counts
outliers = daily_counts[(daily_counts['daily_count'] < lower_bound) | (daily_counts['daily_count'] > upper_bound)]


In [107]:
nan_rows_arousal = df[(df['variable'] == 'circumplex.arousal') & (df['value'].isna())].index
nan_rows_valence = df[(df['variable'] == 'circumplex.valence') & (df['value'].isna())].index
nan_rows_activity = df[(df['variable'] == 'activity') & (df['value'].isna())].index


# Combine the indices of rows with NaN values for arousal and valence
nan_rows_combined = nan_rows_arousal.union(nan_rows_valence)

In [108]:
# List of variables/categories to check for negative values, excluding mood, arousal, and valence
variables_to_check = [variable for variable in df['variable'].unique() if variable not in ['mood', 'circumplex.arousal', 'circumplex.valence']]

# Check for negative values in the remaining variables
negative_values_check = {variable: (df[df['variable'] == variable]['value'] < 0).any() for variable in variables_to_check}

negative_values_check

# Identify rows with negative values in appCat.builtin and appCat.entertainment in the original dataset
negative_values_builtin = df[(df['variable'] == 'appCat.builtin') & (df['value'] < 0)].index
negative_values_entertainment = df[(df['variable'] == 'appCat.entertainment') & (df['value'] < 0)].index

#neg combined 
neg = negative_values_builtin.union(negative_values_entertainment)
# Combine the indices of rows with negative values for appCat.builtin and appCat.entertainment
# with previously identified NaN rows for removal
remove_combined = nan_rows_combined.union(negative_values_builtin).union(negative_values_entertainment)

df_negative = df.loc[neg]

In [109]:
combined = df.loc[remove_combined]
combined
df1 = df.drop(combined.index)

In [110]:
# Initialize a DataFrame to store outliers
outliers_df = pd.DataFrame()

variables_for_box_plots_all = [
    'screen', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment',
    'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.other', 'appCat.social',
    'appCat.travel', 'appCat.unknown', 'appCat.utilities', 'appCat.weather'
]
# Iterate over each variable to remove outliers, saving them first
for variable in variables_for_box_plots_all:
    # Isolate the current variable's data
    var_df = df1[df1['variable'] == variable]
    
    # Calculate IQR and determine bounds for outliers
    Q1 = var_df['value'].quantile(0.25)
    Q3 = var_df['value'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identifying outliers
    outliers = var_df[(var_df['value'] < lower_bound) | (var_df['value'] > upper_bound)]
    
    # Append outliers to the outliers_df DataFrame
    outliers_df = pd.concat([outliers_df, outliers], ignore_index=True)
    
    # Identifying indexes of rows that are not outliers to keep in the original dataframe
    non_outliers_index = var_df[(var_df['value'] >= lower_bound) & (var_df['value'] <= upper_bound)].index
    
    # Update df to only include rows that are not outliers for the current variable
    df1 = df1[(df1.index.isin(non_outliers_index)) | (df1['variable'] != variable)]


In [111]:
score_variables = ["mood", "circumplex.arousal", "circumplex.valence", "activity"]

# Creating a dataset with only the selected variables
df_score = df1[df1['variable'].isin(score_variables)]

# Creating another dataset with the rest of the variables
df_machine = df1[~df1['variable'].isin(score_variables)]

df_score['date'] = df_score['time'].dt.date
df_machine['date'] = df_machine['time'].dt.date


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/3639623860.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_score['date'] = df_score['time'].dt.date
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/3639623860.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_machine['date'] = df_machine['time'].dt.date


In [112]:
df_scores_daily = df_score.drop('time', axis = 1)
df_machine_daily = df_machine.drop('time', axis = 1)

In [113]:
grouped_scores = df_scores_daily.groupby(['id', 'date', 'variable'])['value'].mean().reset_index()
grouped_scores
grouped_times = df_machine_daily.groupby(['id', 'date', 'variable'])['value'].sum().reset_index()
grouped_times

,id,date,variable,value
0,AS14.01,2014-02-17,call,2.000000
1,AS14.01,2014-02-18,call,1.000000
2,AS14.01,2014-02-19,call,7.000000
3,AS14.01,2014-02-19,sms,2.000000
4,AS14.01,2014-02-20,call,2.000000
...,...,...,...,...
10255,AS14.33,2014-05-30,appCat.travel,915.714000
10256,AS14.33,2014-05-30,appCat.unknown,8.072000
10257,AS14.33,2014-05-30,appCat.utilities,155.922000
10258,AS14.33,2014-05-30,call,4.000000


In [114]:
scores_pivot_df = df_scores_daily.pivot_table(index=['id','date'], columns='variable', values='value', aggfunc='mean').reset_index()
scores_pivot_df

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood
0,AS14.01,2014-02-26,NaN,-0.250000,0.750000,6.250000
1,AS14.01,2014-02-27,NaN,0.000000,0.333333,6.333333
2,AS14.01,2014-03-20,0.081548,NaN,NaN,NaN
3,AS14.01,2014-03-21,0.134050,0.200000,0.200000,6.200000
4,AS14.01,2014-03-22,0.236880,0.600000,0.500000,6.400000
...,...,...,...,...,...,...
1301,AS14.33,2014-05-27,0.012704,-0.600000,0.400000,6.200000
1302,AS14.33,2014-05-28,0.103301,0.000000,1.200000,8.200000
1303,AS14.33,2014-05-29,0.169354,-1.333333,1.000000,7.000000
1304,AS14.33,2014-05-30,0.192901,-0.800000,-0.400000,6.800000


In [115]:
time_pivot_df = df_machine_daily.pivot_table(index=['id','date'], columns='variable', values='value', aggfunc='sum').reset_index()

In [116]:
app_categories_columns = [
    "appCat.builtin", "appCat.communication", "appCat.entertainment",
    "appCat.finance", "appCat.game", "appCat.office", "appCat.other",
    "appCat.social", "appCat.travel", "appCat.unknown", "appCat.utilities",
    "appCat.weather"
]

# Ensure the DataFrame has these columns; this prevents KeyError if some columns don't exist
existing_app_columns = [col for col in app_categories_columns if col in time_pivot_df.columns]

# Replace NaN values with 0 for the specified app category columns
time_pivot_df[existing_app_columns] = time_pivot_df[existing_app_columns].fillna(0)
time_pivot_df



variable,id,date,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-17,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,2.0,NaN,NaN
1,AS14.01,2014-02-18,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,1.0,NaN,NaN
2,AS14.01,2014-02-19,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,7.0,NaN,2.0
3,AS14.01,2014-02-20,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,2.0,NaN,3.0
4,AS14.01,2014-02-21,0.000,0.000,0.000,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.0,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,AS14.33,2014-05-26,571.902,750.571,79.963,0.0,0.0,0.000,140.446,1508.191,0.000,0.000,0.000,0.0,10.0,1118.782999,3.0
1911,AS14.33,2014-05-27,269.550,771.182,32.137,0.0,0.0,0.000,57.535,1252.079,0.000,0.000,56.173,0.0,1.0,1508.516001,2.0
1912,AS14.33,2014-05-28,1093.741,1057.568,153.121,0.0,0.0,38.262,208.962,1817.695,0.000,0.000,30.666,0.0,10.0,3414.742998,1.0
1913,AS14.33,2014-05-29,248.063,198.112,5.018,0.0,0.0,0.000,29.202,377.179,0.939,0.000,3.199,0.0,5.0,395.046000,1.0


In [117]:
merged_df = pd.merge(scores_pivot_df, time_pivot_df, on=['id','date'], how='inner')
merged_df

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,NaN,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.0
1,AS14.01,2014-03-20,0.081548,NaN,NaN,NaN,121.466,980.911,42.593,0.000,...,0.000,11.345,621.094,0.000,45.173,21.074,0.000,1.0,128.843000,NaN
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,NaN
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.0
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,NaN,2780.277001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.0
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.0
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.0
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.0


NOW SPLIT INTO TRAIN TEST SETS, THEN ADD ALL INTERPOLATION STUFF TO BOTH

In [118]:
#tscv = TimeSeriesSplit(n_splits=2) #ONLY 1 SPLIT = change to 5 for kfoldcross
train_df = merged_df

In [119]:
columns_to_interpolate = ['call',  'sms']
train_df[columns_to_interpolate] = train_df[columns_to_interpolate].interpolate(method='linear', limit_direction='forward', axis=0)
mean_activity_per_id = train_df.groupby('id')['activity'].mean()
train_df['activity'] = train_df.apply(
    lambda row: mean_activity_per_id[row['id']] if pd.isna(row['activity']) else row['activity'],
    axis=1
)
#test_df[columns_to_interpolate] = test_df[columns_to_interpolate].interpolate(method='linear', limit_direction='forward', axis=0)
train_df

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
1,AS14.01,2014-03-20,0.081548,NaN,NaN,NaN,121.466,980.911,42.593,0.000,...,0.000,11.345,621.094,0.000,45.173,21.074,0.000,1.0,128.843000,1.666667
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [120]:
columns_to_interpolate = ['call', 'sms', 'activity']
train_df[columns_to_interpolate] = train_df.groupby('id')[columns_to_interpolate].transform(lambda group: group.interpolate(method='linear', limit_direction='forward', axis=0))
#test_df[columns_to_interpolate] = test_df.groupby('id')[columns_to_interpolate].transform(lambda group: group.interpolate(method='linear', limit_direction='forward', axis=0))

train_df

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
1,AS14.01,2014-03-20,0.081548,NaN,NaN,NaN,121.466,980.911,42.593,0.000,...,0.000,11.345,621.094,0.000,45.173,21.074,0.000,1.0,128.843000,1.666667
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [121]:
columns_to_check = ['mood']


train_df_cleaned = train_df.dropna(subset=columns_to_check)
#test_df = test_df.dropna(subset=columns_to_check)
train_df_cleaned

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,3.010,66.558,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [122]:
not_null_columns = [
    "circumplex.valence"
]

# Ensure the DataFrame has these columns; this prevents KeyError if some columns don't exist
existing_app_columns = [col for col in not_null_columns if col in train_df_cleaned.columns]

# Replace NaN values with 0 for the specified app category columns
train_df_cleaned[existing_app_columns] = train_df_cleaned[existing_app_columns].fillna(0)
#test_df[existing_app_columns] = test_df[existing_app_columns].fillna(0)
train_df_cleaned


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/4185145295.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned[existing_app_columns] = train_df_cleaned[existing_app_columns].fillna(0)


variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,3.010,66.558,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [123]:
app_categories_columns = [
    "appCat.builtin", "appCat.communication", "appCat.entertainment",
    "appCat.finance", "appCat.game", "appCat.office", "appCat.other",
    "appCat.social", "appCat.travel", "appCat.unknown", "appCat.utilities",
    "appCat.weather",
]

# Ensure the DataFrame has these columns; this prevents KeyError if some columns don't exist
existing_app_columns = [col for col in app_categories_columns if col in train_df_cleaned.columns]

# Replace NaN values with 0 for the specified app category columns
train_df_cleaned[existing_app_columns] = train_df_cleaned[existing_app_columns].fillna(0)
#test_df[existing_app_columns] = test_df[existing_app_columns].fillna(0)
train_df_cleaned


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/1844166882.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned[existing_app_columns] = train_df_cleaned[existing_app_columns].fillna(0)


variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,NaN,2.000000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,3.010,66.558,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [124]:
columns_to_sum = [
    "appCat.builtin", "appCat.communication", "appCat.entertainment",
    "appCat.finance", "appCat.game", "appCat.office", "appCat.other",
    "appCat.social", "appCat.travel", "appCat.unknown", "appCat.utilities",
    "appCat.weather"
]

# Use apply to iterate over rows for rows where 'screen' is NaN
train_df_cleaned.loc[train_df_cleaned['screen'].isna(), 'screen'] = train_df_cleaned[train_df_cleaned['screen'].isna()].apply(
    lambda row: row[columns_to_sum].sum(), axis=1)
#test_df.loc[test_df['screen'].isna(), 'screen'] = test_df[test_df['screen'].isna()].apply(
    #lambda row: row[columns_to_sum].sum(), axis=1)

train_df_cleaned

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms
0,AS14.01,2014-02-26,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.0,0.000000,2.000000
2,AS14.01,2014-03-21,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.479,50.465,3121.747,167.373,0.000,229.138,0.000,6.0,2867.963000,1.333333
3,AS14.01,2014-03-22,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.000,60.565,439.632,37.305,0.000,53.188,0.000,3.0,3038.464000,1.000000
4,AS14.01,2014-03-23,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.000,40.604,649.414,0.000,0.000,30.086,30.386,6.5,2780.277001,1.000000
5,AS14.01,2014-03-24,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,3.010,66.558,1362.785,35.005,0.000,10.064,0.000,10.0,5953.697001,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,AS14.33,2014-05-26,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000,140.446,1508.191,0.000,0.000,0.000,0.000,10.0,1118.782999,3.000000
1244,AS14.33,2014-05-27,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000,57.535,1252.079,0.000,0.000,56.173,0.000,1.0,1508.516001,2.000000
1245,AS14.33,2014-05-28,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,38.262,208.962,1817.695,0.000,0.000,30.666,0.000,10.0,3414.742998,1.000000
1246,AS14.33,2014-05-29,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000,29.202,377.179,0.939,0.000,3.199,0.000,5.0,395.046000,1.000000


In [125]:
# def custom_3day_avg(series):

#     result = np.empty_like(series, dtype=float)
#     result[:] = np.nan  # Initialize with NaN
    
#     # Iterate over the series using index for access
#     for i in range(len(series)):
#         if i == 0:
#             result[i] = np.nan  # First day is NaN
#         elif i == 1:
#             result[i] = series.iloc[i-1]  # Second day is the value of the first day
#         elif i == 2:
#             result[i] = (series.iloc[i-2] + series.iloc[i-1]) / 2  # Third day is the average of the first two days
#         else:
#             result[i] = (series.iloc[i-3] + series.iloc[i-2] + series.iloc[i-1]) / 3  # From the fourth day, use a regular 3-day average
    
#     return pd.Series(result, index=series.index)
    
# train_df_cleaned['3_day_avg_mood'] = train_df_cleaned.groupby('id')['mood'].transform(custom_3day_avg)
# train_df_cleaned['3_day_avg_activity'] = train_df_cleaned.groupby('id')['activity'].transform(custom_3day_avg)
# train_df_cleaned['3_day_avg_mood'] = train_df_cleaned.groupby('id')['3_day_avg_mood'].fillna(method='ffill')
# train_df_cleaned['3_day_avg_activity'] = train_df_cleaned.groupby('id')['3_day_avg_activity'].fillna(method='ffill')
# #test_df['3_day_avg_mood'] = test_df.groupby('id')['mood'].transform(custom_3day_avg)
# #test_df['3_day_avg_activity'] = test_df.groupby('id')['activity'].transform(custom_3day_avg)
# #test_df['3_day_avg_mood'] = test_df.groupby('id')['3_day_avg_mood'].fillna(method='ffill')
# #test_df['3_day_avg_activity'] = test_df.groupby('id')['3_day_avg_activity'].fillna(method='ffill')
# mean_mood_per_id = train_df.groupby('id')['mood'].mean()
# train_df_cleaned['3_day_avg_mood'] = train_df_cleaned.apply(
#     lambda row: mean_mood_per_id[row['id']] if pd.isna(row['3_day_avg_mood']) else row['3_day_avg_mood'],
#     axis=1
# )
# mean_activity_per_id = train_df.groupby('id')['activity'].mean()
# train_df_cleaned['3_day_avg_activity'] = train_df_cleaned.apply(
#     lambda row: mean_activity_per_id[row['id']] if pd.isna(row['3_day_avg_activity']) else row['3_day_avg_activity'],
#     axis=1
# )
# train_df_cleaned


In [126]:
# def custom_3day_avg(series, shift=True):
#     result = np.empty_like(series, dtype=float)
#     result[:] = np.nan  # Initialize with NaN
    
#     if shift:
#         # Shifted averaging for mood to prevent data leakage
#         for i in range(len(series)):
#             if i == 0:
#                 result[i] = np.nan  # First day is NaN
#             elif i == 1:
#                 result[i] = series.iloc[i-1]  # Second day is the value of the first day
#             elif i == 2:
#                 result[i] = (series.iloc[i-2] + series.iloc[i-1]) / 2  # Third day is the average of the first two days
#             else:
#                 result[i] = (series.iloc[i-3] + series.iloc[i-2] + series.iloc[i-1]) / 3  # Regular 3-day average
#     else:
#         # Non-shifted averaging for other metrics
#         for i in range(len(series)):
#             if i == 0:
#                 result[i] = series.iloc[i]  # First day is NaN
#             elif i == 1:
#                 result[i] = series.iloc[i]+ series.iloc[i-1]  # Second day is the value of the first day
#             else:
#                 result[i] = (series.iloc[i-2] + series.iloc[i-1] + series.iloc[i]) / 3
    
#     return pd.Series(result, index=series.index)

# # Assume 'train_df' is your DataFrame with all the columns loaded
# columns_to_average = [
#     "activity", "circumplex.arousal", "circumplex.valence", "mood",
#     "appCat.builtin", "appCat.communication", "appCat.entertainment",
#     "appCat.finance", "appCat.office", "appCat.other", "appCat.social",
#     "appCat.travel", "appCat.unknown", "appCat.utilities", "appCat.weather",
#     "call", "screen", "sms"
# ]

# # Apply the function to each column
# for column in columns_to_average:
#     shift = True if column == 'mood' else False
#     train_df_cleaned['3_day_avg_' + column] = train_df_cleaned.groupby('id')[column].transform(lambda x: custom_3day_avg(x, shift=shift))

# # If some rows are still NaN after group transformation (e.g., early entries for each group), fill with the mean from that user
# for column in columns_to_average:
#     mean_per_id = train_df_cleaned.groupby('id')[column].mean()
#     train_df_cleaned['3_day_avg_' + column] = train_df_cleaned.apply(
#         lambda row: mean_per_id[row['id']] if pd.isna(row['3_day_avg_' + column]) else row['3_day_avg_' + column],
#         axis=1
#     )


In [127]:
# def custom_7day_avg(series, shift=True):
#     result = np.empty_like(series, dtype=float)
#     result[:] = np.nan  # Initialize with NaN
    
#     if shift:
#         # Shifted averaging for mood to prevent data leakage
#         for i in range(len(series)):
#             if i == 0:
#                 result[i] = np.nan  # First day is NaN
#             elif i <= 6:
#                 # Average only up to the number of available days, excluding the current day
#                 result[i] = np.mean(series.iloc[:i])
#             else:
#                 # Average the past 7 days, excluding the current day
#                 result[i] = np.mean(series.iloc[i-7:i])
#     else:
#         # Non-shifted averaging for other metrics
#         for i in range(len(series)):
#             if i == 0:
#                 result[i] = series.iloc[i] # First day is NaN
#             elif i <= 6:
#                 # Average including the current day and as many days before as available
#                 result[i] = np.mean(series.iloc[:i+1])
#             else:
#                 # Regular 7-day average including the current day
#                 result[i] = np.mean(series.iloc[i-6:i+1])
    
#     return pd.Series(result, index=series.index)
# columns_to_average = [
#     "activity", "circumplex.arousal", "circumplex.valence", "mood",
#     "appCat.builtin", "appCat.communication", "appCat.entertainment",
#     "appCat.finance", "appCat.office", "appCat.other", "appCat.social",
#     "appCat.travel", "appCat.unknown", "appCat.utilities", "appCat.weather",
#     "call", "screen", "sms"
# ]

# # Apply the function to each column
# for column in columns_to_average:
#     shift = True if column == 'mood' else False
#     train_df_cleaned['7_day_avg_' + column] = train_df_cleaned.groupby('id')[column].transform(lambda x: custom_7day_avg(x, shift=shift))

# # If some rows are still NaN after group transformation (e.g., early entries for each group), fill with the mean from that user
# for column in columns_to_average:
#     mean_per_id = train_df_cleaned.groupby('id')[column].mean()
#     train_df_cleaned['7_day_avg_' + column] = train_df_cleaned.apply(
#         lambda row: mean_per_id[row['id']] if pd.isna(row['7_day_avg_' + column]) else row['7_day_avg_' + column],
#         axis=1
#     )

In [128]:
import pandas as pd

# Assuming train_df_cleaned is already loaded and prepared
# Define weights for each category - adjust these based on your specific requirements
weights = {
    'appCat.finance': 0.5,      # Weight for finance apps
    'appCat.office': 0.5,       # Weight for office apps
    'appCat.communication': 0.5,# Weight for communication apps
    'appCat.social': 0.5,       # Weight for social apps
    'appCat.entertainment': 0.5 # Weight for entertainment apps
}

# Calculate weighted sums
train_df_cleaned['productivity_attribute'] = (
    train_df_cleaned['appCat.finance'] * weights['appCat.finance'] +
    train_df_cleaned['appCat.office'] * weights['appCat.office']
)

train_df_cleaned['social_app'] = (
    train_df_cleaned['appCat.communication'] * weights['appCat.communication'] +
    train_df_cleaned['appCat.social'] * weights['appCat.social']
)

# Calculate weighted ratio for productivity to social apps
# Adding 1 to avoid division by zero in case totals are zero
train_df_cleaned['productivity_to_social_app_ratio'] = (
    (train_df_cleaned['appCat.finance'] * weights['appCat.finance'] +
    train_df_cleaned['appCat.office'] * weights['appCat.office'] + 1) /
    (train_df_cleaned['appCat.entertainment'] * weights['appCat.entertainment'] +
    train_df_cleaned['appCat.social'] * weights['appCat.social'] + 1)
)

# Assuming sms and call are equally important for 'social_phone'
train_df_cleaned['social_phone'] = (
    train_df_cleaned['sms'] * 0.5 +  # Assuming equal weight for sms
    train_df_cleaned['call'] * 0.5   # Assuming equal weight for calls
)

# Display the modified DataFrame
train_df_cleaned.head()


/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/2860325050.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned['productivity_attribute'] = (
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/2860325050.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_cleaned['social_app'] = (
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/2860325050.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

variable,id,date,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms,productivity_attribute,social_app,productivity_to_social_app_ratio,social_phone
0,AS14.01,2014-02-26,0.090464,-0.25,0.75,6.25,0.000,0.000,0.000,0.000,...,0.0,0.000,0.000,1.0,0.000000,2.000000,0.0000,0.0000,1.000000,1.500000
2,AS14.01,2014-03-21,0.134050,0.20,0.20,6.20,1090.617,2850.042,100.898,49.544,...,0.0,229.138,0.000,6.0,2867.963000,1.333333,27.5115,2985.8945,0.017683,3.666667
3,AS14.01,2014-03-22,0.236880,0.60,0.50,6.40,476.737,1946.683,4.010,21.076,...,0.0,53.188,0.000,3.0,3038.464000,1.000000,10.5380,1193.1575,0.051781,2.000000
4,AS14.01,2014-03-23,0.142741,0.20,0.80,6.80,540.915,2680.535,4.028,43.403,...,0.0,30.086,30.386,6.5,2780.277001,1.000000,21.7015,1664.9745,0.069271,3.750000
5,AS14.01,2014-03-24,0.078961,0.80,0.00,6.00,731.456,5070.628,70.289,34.106,...,0.0,10.064,0.000,10.0,5953.697001,1.000000,18.5580,3216.7065,0.027257,5.500000


In [129]:
def calculate_moving_averages(df):
    df = df.set_index('date')
    
    # Convert all columns to numeric, ignoring errors to skip non-convertible columns
    numeric_df = df.apply(pd.to_numeric, errors='ignore')

    # Select only the numeric columns for rolling mean calculation
    numeric_cols = numeric_df.select_dtypes(include=['number'])

    # Calculate moving averages with shifting to avoid data leakage
    ma_3days = numeric_cols.rolling(window=3).mean().shift(1)
    ma_7days = numeric_cols.rolling(window=7).mean().shift(1)

    # Add suffixes to identify the columns for each moving average
    ma_3days.columns = [f"{col}_3day_avg" for col in ma_3days.columns]
    ma_7days.columns = [f"{col}_7day_avg" for col in ma_7days.columns]

    # Concatenate the original numeric data with the moving averages
    result = pd.concat([numeric_df, ma_3days, ma_7days], axis=1)

    # Handle NaN values: forward fill first, then backward fill
    result.ffill(inplace=True)
    result.bfill(inplace=True)
    
    return result.reset_index()  # Reset the index to bring 'date' back to a column

# Assuming 'train_df_cleaned' is your DataFrame and 'id' is the group identifier
train_df_cleaned = train_df_cleaned.groupby('id').apply(calculate_moving_averages).reset_index(drop=True)

/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/3960753109.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  numeric_df = df.apply(pd.to_numeric, errors='ignore')
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/3960753109.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  numeric_df = df.apply(pd.to_numeric, errors='ignore')
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/3960753109.py:5: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  numeric_df = df.apply(pd.to_numeric, errors='ignore')
/var/folders/_1/g4tvh5tn2wb8t1qyc72n4cdr0000gn/T/ipykernel_48442/3960753109.py:5: FutureWarning: errors

In [130]:
train_df_cleaned['productivity_to_social_ratio'] = (train_df_cleaned['appCat.finance'] + train_df_cleaned['appCat.office']) +1/ (train_df_cleaned['appCat.entertainment'] + train_df_cleaned['appCat.social']+1)
train_df_cleaned['productivity_attribute'] = train_df_cleaned['appCat.finance'] + train_df_cleaned['appCat.office']
train_df_cleaned['social_attribute'] = train_df_cleaned['appCat.entertainment'] + train_df_cleaned['appCat.social']
#test_df['productivity_to_social_ratio'] = (test_df['appCat.finance'] + test_df['appCat.office']) +1/ (test_df['appCat.entertainment'] + test_df['appCat.social']+1)
#test_df['productivity_attribute'] = test_df['appCat.finance'] + test_df['appCat.office']
#test_df['social_attribute'] = test_df['appCat.entertainment'] + test_df['appCat.social']
train_df_cleaned

,date,id,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,appCat.weather_7day_avg,call_7day_avg,screen_7day_avg,sms_7day_avg,productivity_attribute_7day_avg,social_app_7day_avg,productivity_to_social_app_ratio_7day_avg,social_phone_7day_avg,productivity_to_social_ratio,social_attribute
0,2014-02-26,AS14.01,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,4.340857,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,1.000000,0.000
1,2014-03-21,AS14.01,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,4.340857,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,55.023310,3222.645
2,2014-03-22,AS14.01,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,4.340857,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,21.078249,443.642
3,2014-03-23,AS14.01,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,4.340857,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,43.404528,653.442
4,2014-03-24,AS14.01,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,4.340857,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,37.116697,1433.074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,2014-05-26,AS14.33,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,0.000000,3.385714,1288.826286,1.000000,0.000000,1066.217214,0.001597,2.192857,0.000629,1588.154
1208,2014-05-27,AS14.33,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,0.000000,4.300000,1342.314428,1.285714,0.000000,1144.606214,0.001498,2.792857,0.000778,1284.216
1209,2014-05-28,AS14.33,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,0.000000,3.814286,1487.639000,1.428571,0.000000,1209.835786,0.001414,2.621429,38.262507,1970.816
1210,2014-05-29,AS14.33,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,0.000000,4.500000,1823.659142,1.428571,2.733000,1291.393500,0.004150,2.964286,0.002610,382.197


In [131]:
# Assuming 'df' is your DataFrame and 'grade' is your continuous variable
train_df_cleaned['mood_quantiles'] = pd.qcut(train_df_cleaned['mood'], q=3, labels=['Q1', 'Q2', 'Q3'])
#test_df['mood_quantiles'] = pd.qcut(test_df['mood'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
train_df_cleaned


,date,id,activity,circumplex.arousal,circumplex.valence,mood,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,...,call_7day_avg,screen_7day_avg,sms_7day_avg,productivity_attribute_7day_avg,social_app_7day_avg,productivity_to_social_app_ratio_7day_avg,social_phone_7day_avg,productivity_to_social_ratio,social_attribute,mood_quantiles
0,2014-02-26,AS14.01,0.090464,-0.250000,0.75,6.25,0.000,0.000,0.000,0.000,...,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,1.000000,0.000,Q1
1,2014-03-21,AS14.01,0.134050,0.200000,0.20,6.20,1090.617,2850.042,100.898,49.544,...,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,55.023310,3222.645,Q1
2,2014-03-22,AS14.01,0.236880,0.600000,0.50,6.40,476.737,1946.683,4.010,21.076,...,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,21.078249,443.642,Q1
3,2014-03-23,AS14.01,0.142741,0.200000,0.80,6.80,540.915,2680.535,4.028,43.403,...,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,43.404528,653.442,Q1
4,2014-03-24,AS14.01,0.078961,0.800000,0.00,6.00,731.456,5070.628,70.289,34.106,...,5.500000,3298.692857,1.190476,18.000214,2072.657286,0.176490,3.345238,37.116697,1433.074,Q1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1207,2014-05-26,AS14.33,0.115201,-0.400000,0.00,5.40,571.902,750.571,79.963,0.000,...,3.385714,1288.826286,1.000000,0.000000,1066.217214,0.001597,2.192857,0.000629,1588.154,Q1
1208,2014-05-27,AS14.33,0.012704,-0.600000,0.40,6.20,269.550,771.182,32.137,0.000,...,4.300000,1342.314428,1.285714,0.000000,1144.606214,0.001498,2.792857,0.000778,1284.216,Q1
1209,2014-05-28,AS14.33,0.103301,0.000000,1.20,8.20,1093.741,1057.568,153.121,0.000,...,3.814286,1487.639000,1.428571,0.000000,1209.835786,0.001414,2.621429,38.262507,1970.816,Q3
1210,2014-05-29,AS14.33,0.169354,-1.333333,1.00,7.00,248.063,198.112,5.018,0.000,...,4.500000,1823.659142,1.428571,2.733000,1291.393500,0.004150,2.964286,0.002610,382.197,Q2


In [132]:
train_df_cleaned.to_csv('out.csv') 
#test_df.to_csv('test.csv')

#merged_df_2 = pd.concat([train_df_cleaned, test_df])
#merged_df_2.sort_values(by=['id', 'date'])
##merged_df_2.to_csv("out.csv")
train_df_cleaned.isnull().sum()

date                                         0
id                                           0
activity                                     0
circumplex.arousal                           0
circumplex.valence                           0
                                            ..
productivity_to_social_app_ratio_7day_avg    0
social_phone_7day_avg                        0
productivity_to_social_ratio                 0
social_attribute                             0
mood_quantiles                               0
Length: 74, dtype: int64